In [4]:
library(moments); library(Rfast)

In [27]:
norm <- readRDS('../data/family_datasets/data_norm/AE3_scran_norm.rds')
load('../data/family_datasets/family_info/family_info_AE3_nocellcyclesplit.RData')
#mom <- load('../data/family_datasets/data_norm/Momentum_AE3_nocellcyclesplit.RData')
intron <- load('../data/Momenti_and_introns_all_datasets/Introns_AE3.RData')

#Get the cells of interest (belonging to families of 3 to 5 members)
family_info = family_info_1[as.character(family_info_1$cell.barcode) %in% colnames(norm),]
families_of_interest = names(table(family_info$clone.id))[(table(family_info$clone.id)>=3)&(table(family_info$clone.id)<=5)]
family_dict = family_info$clone.id; names(family_dict) = family_info$cell.barcode

In [37]:
family_genes_correlation <- function(data_matrix, family_dict, families_of_interest)
{
  #Get genes name + cells of interest
  genes = rownames(data_matrix)
  family_dict = family_dict[(names(family_dict) %in% colnames(data_matrix)) & (family_dict %in% families_of_interest)]
  
  #Construct matrix  (#cells x #cells), at intersection of two cells if T -> belong to same family, set diagonal + lower tri to F
    real_family_matrix = outer(family_dict[colnames(data_matrix)], family_dict[colnames(data_matrix)], FUN='=='); diag(real_family_matrix) = F; real_family_matrix[lower.tri(real_family_matrix)] = F
  #Get the indices of the cells belonging to same family
  tmp = which(real_family_matrix, arr.ind=T)
  family_members_a = colnames(data_matrix)[tmp[,1]]; family_members_b = colnames(data_matrix)[tmp[,2]]
    
  #Compute family correlation for each gene
  family_corr = rep(NA, length(genes)); names(family_corr) = genes
  for (i in seq(1,length(genes))){
      gene = genes[i]; 
      family_corr[gene] = cor(data_matrix[gene,family_members_a], data_matrix[gene,family_members_b], method="spearman")
  }
  return(family_corr[order(family_corr, decreasing=T)])
}

In [38]:
#Only keep the cells of interest in the exon norm data and genes whose expression don't sum up to 0
exons_famfilt = norm[,intersect(names(family_dict)[family_dict %in% families_of_interest], colnames(norm))]; 
exons_famfilt = exons_famfilt[rowSums(exons_famfilt)!=0,]

#Only keep the cells of interest in the momentum data
#moments_famfilt = momentum[,intersect(names(family_dict)[family_dict %in% families_of_interest], colnames(momentum))]

#Only keep the cells of interest in the intron norm data and genes whose expression don't sum up to 0
#introns_famfilt = intron[,intersect(names(family_dict)[family_dict %in% families_of_interest], colnames(intron))]; 
#introns_famfilt = introns_famfilt[rowSums(introns_famfilt)!=0,]

#Compute the gene correlation for exons, introns, and momentum
exon_markers = family_genes_correlation(exons_famfilt, family_dict, families_of_interest)
#moment_markers = family_genes_correlation(moments_famfilt, family_dict, families_of_interest)
#intron_markers = family_genes_correlation(introns_famfilt, family_dict, families_of_interest)

Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, family_members_b], :
"the standard deviation is zero"Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, family_members_b], :
"the standard deviation is zero"Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, family_members_b], :
"the standard deviation is zero"Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, family_members_b], :
"the standard deviation is zero"Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, family_members_b], :
"the standard deviation is zero"Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, family_members_b], :
"the standard deviation is zero"Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, family_members_b], :
"the standard deviation is zero"Warning message in cor(data_matrix[gene, family_members_a], data_matrix[gene, famil

In [40]:
#Get the median expression of the genes
load(file="../data/characteristic_data/master_matrix_Almut.RData")
load(file="../data/characteristic_data/master_matrix_no_NA_Almut.RData")
expression_vector = master_matrix_no_NA$median_WT; names(expression_vector) = rownames(master_matrix_no_NA)

In [43]:
#Compute the skewnesses of all gene by using the norm data
skewnesses = list(); for (gene in names(exon_markers)){skewnesses[gene] = skewness(exons_famfilt[gene,])}

#Get name of genes with particular skewness behaviour
markers_1 = names(exon_markers[1:1000][skewnesses[1:1000]<1])
markers_2 = names(exon_markers[1:1000][skewnesses[1:1000]>3])
markers_3 = names(exon_markers[1:1000][skewnesses[1:1000]>1 and skewnesses[1:1000]<3])

control_1 = sample_constraint_to_many(all = t(master_matrix_no_NA[,c("GC","length","median_WT")]), subset = intersect(markers_1, rownames(master_matrix_no_NA)), 1000, cutoff = 0.75)
control_2 = sample_constraint_to_many(all = t(master_matrix_no_NA[,c("GC","length","median_WT")]), subset = intersect(markers_2, rownames(master_matrix_no_NA)), 1000, cutoff = 0.75)
control_3 = sample_constraint_to_many(all = t(master_matrix_no_NA[,c("GC","length","median_WT")]), subset = intersect(markers_3, rownames(master_matrix_no_NA)), 1000, cutoff = 0.75)

pdf(file='EPFL_chracteristics_skewness_split_ctrls.pdf', width=10, height=10)
par(mfrow=c(3,3))
measures = c("GC", "length", "half_life_WT", "ATAC_1", "pbody", "noise_WT", "protein", "WT_bulk", "protein_half_life_1",
             "Srebf1", "Myc", "Nanog", "Dnmt3a", "Sox2", "H3K27me3", "H3K4me3", "H3K27ac", "H3K9me3")
for (measure in measures)
{
  boxplot(master_matrix[master_matrix$ensembl_ID %in% markers_1,measure],
          master_matrix[master_matrix$ensembl_ID %in% control_1,measure],
          master_matrix[master_matrix$ensembl_ID %in% markers_2,measure],
          master_matrix[master_matrix$ensembl_ID %in% control_2,measure],
          master_matrix[,measure],
          names = c('mark.\nquant.','ctrls\nquant.','mark.\nqual.','ctrls\nqual.','all'), col=base_colors, outline=F, main=measure, frame.plot=F, las=2, main="exons")
}
dev.off()

ERROR: Error in sample_constraint_to_many(all = t(master_matrix_no_NA[, c("GC", : could not find function "sample_constraint_to_many"


In [ ]:
#Get genes with intron gene and momentum biggest correlation
markers_1 = names(moment_markers[1:500])
markers_2 = names(intron_markers[1:500])
control_1 = sample_constraint_to_many(all = t(master_matrix_no_NA[,c("GC","length","median_WT")]), subset = intersect(markers_1, rownames(master_matrix_no_NA)), 500, cutoff = 0.5)
control_2 = sample_constraint_to_many(all = t(master_matrix_no_NA[,c("GC","length","median_WT")]), subset = intersect(markers_2, rownames(master_matrix_no_NA)), 500, cutoff = 0.5)

pdf(file='EPFL_chracteristics_introns_and_moments_ctrls.pdf', width=10, height=10)
par(mfrow=c(3,3))
measures = c("GC", "length", "half_life_WT", "ATAC_1", "pbody", "noise_WT", "protein", "WT_bulk", "protein_half_life_1",
             "Srebf1", "Myc", "Nanog", "Dnmt3a", "Sox2", "H3K27me3", "H3K4me3", "H3K27ac", "H3K9me3")
for (measure in measures)
{
  boxplot(master_matrix[master_matrix$ensembl_ID %in% markers_1,measure],
          master_matrix[master_matrix$ensembl_ID %in% control_1,measure],
          master_matrix[master_matrix$ensembl_ID %in% markers_2,measure],
          master_matrix[master_matrix$ensembl_ID %in% control_2,measure],
          master_matrix[,measure],
          names = c('mark.\nmoments','ctrls\nmoments','mark.\nintrons','ctrls\nintrons','all'), col=base_colors, outline=F, main=measure, frame.plot=F, las=2, main="exons")
}
dev.off()

In [61]:
# gene ontologies
library(biomaRt); library(org.Mm.eg.db); library(topGO)
GO_enrichment <- function(gene_set, background_data, category='BP') # This can be BP, MF or CC
{ 
  #Get the name of background genes and name of genes in gene set
  background_IDs = names(background_data); gene_IDs = gene_set
  all_IDs = rep(1, length(background_IDs)); names(all_IDs) = background_IDs
  all_IDs[gene_IDs] = 0 #Set to zero for genes in gene set
  
  #Create topGOdata object, perform fisher test
  GOdata <- new("topGOdata", ontology = category, allGenes = all_IDs, geneSel = function(p) p < 0.05, description = "Test", annot = annFUN.org, mapping = "org.Mm.eg.db", ID = "Ensembl")
  resultFisher <- runTest(GOdata, algorithm = "classic", statistic = "fisher")
  
  #Put result in a table 
  GO_table <- GenTable(GOdata, classicFisher = resultFisher, topNodes = length(score(resultFisher)))
  gene_table <- lapply(GO_table$GO.ID, function(x) intersect(as.character(unlist(genesInTerm(object = GOdata, whichGO = x))), gene_set))
  gene_table_all  <- lapply(GO_table$GO.ID, function(x) as.character(unlist(genesInTerm(object = GOdata, whichGO = x))))
  rownames(GO_table) = GO_table$GO.ID; GO_table$Enrichment = (GO_table$Significant+0.1) / (GO_table$Expected+0.1)
  
  ctrl_matrix = matrix(NA, ncol=1000, nrow=1000)
  for (i in seq(1,1000))
  {
    tmp_set = sample_constraint_to_one(background_data, gene_set, size=length(gene_set), N=100)
    tmp_list = lapply(gene_table_all[1:1000], function(x) intersect(x, tmp_set))
    ctrl_matrix[,i] = unlist(lapply(tmp_list, function(x) length(x)))
  }
  
  GO_table[1:1000,c("50%","75%","90%","95%","99%","100%")] = round(t(apply(ctrl_matrix, 1, function(x) quantile(x,c(0.5,0.75,0.9,0.95,0.99,1)))))
  
  result = list(); result[['GO_table']] = GO_table; result[['gene_table']] = gene_table; result[['gene_table_all']] = gene_table_all
  return(result)
}

In [ ]:
load('~/Desktop/R_code/go_mapping.RData')

GO_results_ESC_exon_quant = GO_enrichment(intersect(names(exon_markers[1:1000][skewnesses[1:1000]<1]), rownames(master_matrix_no_NA)), expression_vector, category='BP')
GO_results_ESC_exon_qual = GO_enrichment(intersect(names(exon_markers[1:1000][skewnesses[1:1000]>1]), rownames(master_matrix_no_NA)), expression_vector, category='BP')
GO_results_ESC_moment = GO_enrichment(intersect(names(moment_markers[1:500]), rownames(master_matrix_no_NA)), expression_vector, category='BP')
GO_results_ESC_intron = GO_enrichment(intersect(names(intron_markers[1:500]), rownames(master_matrix_no_NA)), expression_vector, category='BP')

GO_results_ESC_exon_quant$GO_table[1:20,]
GO_results_ESC_exon_qual$GO_table[1:20,]
GO_results_ESC_moment$GO_table[1:20,]
GO_results_ESC_intron$GO_table[1:20,]

GO_results_ESC_exon_quant_CC = GO_enrichment(intersect(names(exon_markers[1:1000][skewnesses[1:1000]<1]), rownames(master_matrix_no_NA)), expression_vector, category='CC')
GO_results_ESC_exon_qual_CC = GO_enrichment(intersect(names(exon_markers[1:1000][skewnesses[1:1000]>1]), rownames(master_matrix_no_NA)), expression_vector, category='CC')
GO_results_ESC_moment_CC = GO_enrichment(intersect(names(moment_markers[1:500]), rownames(master_matrix_no_NA)), expression_vector, category='CC')
GO_results_ESC_intron_CC = GO_enrichment(intersect(names(intron_markers[1:500]), rownames(master_matrix_no_NA)), expression_vector, category='CC')

GO_results_ESC_exon_quant_CC$GO_table[1:20,]
GO_results_ESC_exon_qual_CC$GO_table[1:20,]
GO_results_ESC_moment_CC$GO_table[1:20,]
GO_results_ESC_intron_CC$GO_table[1:20,]

In [ ]:
# visualization of some example markers

plot_example <- function(data_matrix, goi, family_dict, fois, main_label=goi)# families with few or no members included
{
  # don't need to make the whole data_matrix
  data_vector = data_matrix[goi,]; names(data_vector) = colnames(data_matrix)
  family_example = matrix(NA, nrow=100, ncol=length(fois)); colnames(family_example) = fois
  for (family in fois)
  {
    cells = names(family_dict)[family_dict==family]
    family_example[,family] = c(as.vector(data_vector[cells]), rep(NA,100-length(cells)))
  }
  sampled_example = matrix(NA, nrow=100, ncol=length(fois)); colnames(sampled_example) = fois
  family_sizes = table(family_dict[family_dict %in% fois])
  for (i in seq(1,length(family_sizes)))
  {
    sampled_families = sample(fois, family_sizes[i]); sampled_cells = c()
    for (family in sampled_families){sampled_cells = c(sampled_cells, sample(names(family_dict)[family_dict==family], 1))}
    sampled_example[,i] = c(as.vector(data_vector[sampled_cells]), rep(NA,100-length(sampled_cells)))
  }
  boxplot(sampled_example[,order(t(rowMedian(t(sampled_example)))+(t(rowMeans(t(sampled_example), na.rm=T))/10**6))]+1, col=rgb(1,1,0,0.33), ylab="Expression", names=NA, main=main_label, outline=F, log='y')
  boxplot(family_example[,order(t(rowMedian(t(family_example)))+(t(rowMeans(t(family_example), na.rm=T))/10**6))]+1, col=rgb(0,0,1,0.33), outline=F, add=T)
}

In [ ]:
plot_example(as.matrix(exons_famfilt), symbols_to_IDs_dict["Rpl32"], family_dict, families_of_interest, main_label="Rpl32"); skewnesses[symbols_to_IDs_dict["Rpl32"]] # quantitative
plot_example(as.matrix(exons_famfilt), symbols_to_IDs_dict["Zscan4d"], family_dict, families_of_interest, main_label="Zscan4d"); skewnesses[symbols_to_IDs_dict["Zscan4d"]] # qualitative

shared_genes = intersect(intersect(names(exon_markers), names(intron_markers)), names(moment_markers))

plot(exon_markers[shared_genes], intron_markers[shared_genes], pch=16, cex=0.5, frame.plot=F)
text(exon_markers[shared_genes], intron_markers[shared_genes], IDs_to_symbols_dict[shared_genes], pos=3)
plot_example(as.matrix(introns_famfilt), symbols_to_IDs_dict["Platr16"], family_dict, families_of_interest, main_label="Platr16") # qualitative: Wnt9a, Drr1, Cd4 | quantitative: Platr16,  Atxn1, Fgf1

plot(intron_markers[shared_genes], moment_markers[shared_genes], pch=16, cex=0.5, frame.plot=F)
text(intron_markers[shared_genes], moment_markers[shared_genes], IDs_to_symbols_dict[shared_genes], pos=3)
plot_example(as.matrix(introns_famfilt), symbols_to_IDs_dict["Pced1b"], family_dict, families_of_interest, main_label="Atxn1") # Pced1b, Prex2, Foxo1, Kat6b